In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
word_vec_dict = {}
with open('../wiki.multi.fr.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.de.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.en.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
danced 37924
seguir 118599
100years 152919
cowes 113501
matanui 77210
elimia 138166
435713
Preparing embedding matrix
Null word embeddings: 4146


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=True,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [3,4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 8
            batch = 64
        else:
            epo = 10
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/muse_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 106380 samples, validate on 53191 samples
Epoch 1/10
106380/106380 [==============================] - 108s 1ms/step - loss: 0.0824 - acc: 0.9752 - val_loss: 0.0477 - val_acc: 0.9821
Epoch 2/10
106380/106380 [==============================] - 107s 1ms/step - loss: 0.0439 - acc: 0.9836 - val_loss: 0.0460 - val_acc: 0.9825
Epoch 3/10
106380/106380 [==============================] - 106s 1ms/step - loss: 0.0362 - acc: 0.9858 - val_loss: 0.0477 - val_acc: 0.9821
Epoch 4/10
106380/106380 [==============================] - 107s 1ms/step - loss: 0.0293 - acc: 0.9884 - val_loss: 0.0551 - val_acc: 0.9825
Epoch 5/10
106380/106380 [==============================] - 107s 1ms/step - loss: 0.0237 - acc: 0.9907 - val_loss: 0.0601 - val_acc: 0.9814
Epoch 6/10
106380/106380 [==============================] - 107s 1ms/step - loss: 0.0188 - acc: 0.9929 - val_loss: 0.0681 - val_acc: 0.9806
Epoch 7/10
106380/106380 [==============================] - 107s 1ms/step - loss: 0.0151 - acc: 0.9944 - val_

-------------------------------
0 0.098775279104 0.963815480256
1 0.0221901343874 0.990486993251
2 0.0498525276666 0.980886251261
3 0.01356563866 0.99700446823
4 0.0667259213135 0.972169128476
5 0.0295329243852 0.991195141974
final 0.0467737375861 0.982592910575
../features/muse_cudnn_gru_3_feat.pkl done
Train on 119678 samples, validate on 39893 samples
Epoch 1/10
119678/119678 [==============================] - 115s 959us/step - loss: 0.0768 - acc: 0.9750 - val_loss: 0.0473 - val_acc: 0.9819
Epoch 2/10
119678/119678 [==============================] - 114s 956us/step - loss: 0.0426 - acc: 0.9839 - val_loss: 0.0456 - val_acc: 0.9829
Epoch 3/10
119678/119678 [==============================] - 114s 953us/step - loss: 0.0347 - acc: 0.9861 - val_loss: 0.0492 - val_acc: 0.9819
Epoch 4/10
119678/119678 [==============================] - 114s 953us/step - loss: 0.0283 - acc: 0.9887 - val_loss: 0.0565 - val_acc: 0.9823
Epoch 5/10
119678/119678 [==============================] - 114s 953us/step

119678/119678 [==============================] - 114s 956us/step - loss: 0.0103 - acc: 0.9962 - val_loss: 0.0935 - val_acc: 0.9795
Epoch 10/10
119678/119678 [==============================] - 114s 953us/step - loss: 0.0083 - acc: 0.9969 - val_loss: 0.1029 - val_acc: 0.9780
Train on 119679 samples, validate on 39892 samples
Epoch 1/10
119679/119679 [==============================] - 115s 960us/step - loss: 0.0841 - acc: 0.9742 - val_loss: 0.0496 - val_acc: 0.9823
Epoch 2/10
119679/119679 [==============================] - 115s 957us/step - loss: 0.0427 - acc: 0.9838 - val_loss: 0.0473 - val_acc: 0.9825
Epoch 3/10
119679/119679 [==============================] - 114s 953us/step - loss: 0.0359 - acc: 0.9861 - val_loss: 0.0506 - val_acc: 0.9821
Epoch 4/10
119679/119679 [==============================] - 114s 953us/step - loss: 0.0294 - acc: 0.9882 - val_loss: 0.0540 - val_acc: 0.9818
Epoch 5/10
119679/119679 [==============================] - 114s 956us/step - loss: 0.0237 - acc: 0.9907 - 

119678/119678 [==============================] - 214s 2ms/step - loss: 0.0279 - acc: 0.9888 - val_loss: 0.0528 - val_acc: 0.9821
Epoch 6/10
119678/119678 [==============================] - 214s 2ms/step - loss: 0.0237 - acc: 0.9906 - val_loss: 0.0620 - val_acc: 0.9824
Epoch 7/10
119678/119678 [==============================] - 213s 2ms/step - loss: 0.0202 - acc: 0.9921 - val_loss: 0.0664 - val_acc: 0.9814
Epoch 8/10
119678/119678 [==============================] - 214s 2ms/step - loss: 0.0170 - acc: 0.9933 - val_loss: 0.0754 - val_acc: 0.9809
Epoch 9/10
119678/119678 [==============================] - 213s 2ms/step - loss: 0.0143 - acc: 0.9945 - val_loss: 0.0825 - val_acc: 0.9808
Epoch 10/10
119678/119678 [==============================] - 213s 2ms/step - loss: 0.0124 - acc: 0.9952 - val_loss: 0.0888 - val_acc: 0.9804
Train on 119678 samples, validate on 39893 samples
Epoch 1/10
119678/119678 [==============================] - 215s 2ms/step - loss: 0.0853 - acc: 0.9743 - val_loss: 0.04

Epoch 8/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0095 - acc: 0.9965 - val_loss: 0.0872 - val_acc: 0.9805
Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0586 - acc: 0.9798 - val_loss: 0.0478 - val_acc: 0.9826
Epoch 2/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0388 - acc: 0.9847 - val_loss: 0.0491 - val_acc: 0.9833
Epoch 3/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0311 - acc: 0.9877 - val_loss: 0.0489 - val_acc: 0.9814
Epoch 4/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0244 - acc: 0.9903 - val_loss: 0.0546 - val_acc: 0.9813
Epoch 5/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0198 - acc: 0.9923 - val_loss: 0.0595 - val_acc: 0.9809
Epoch 6/8
119678/119678 [==============================] - 177s 1ms/step - loss: 0.0156 - acc: 0.9940 - val_loss: 0

Epoch 4/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0316 - acc: 0.9874 - val_loss: 0.0489 - val_acc: 0.9825
Epoch 5/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0270 - acc: 0.9892 - val_loss: 0.0529 - val_acc: 0.9818
Epoch 6/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0228 - acc: 0.9909 - val_loss: 0.0614 - val_acc: 0.9814
Epoch 7/10
106380/106380 [==============================] - 161s 2ms/step - loss: 0.0190 - acc: 0.9926 - val_loss: 0.0725 - val_acc: 0.9812
Epoch 8/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0158 - acc: 0.9939 - val_loss: 0.0725 - val_acc: 0.9799
Epoch 9/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0131 - acc: 0.9950 - val_loss: 0.0868 - val_acc: 0.9807
Epoch 10/10
106380/106380 [==============================] - 160s 2ms/step - loss: 0.0113 - acc: 0.9957 - val_loss: 0.0962 - val_acc: 0.9803
Train on 106381 sam

119678/119678 [==============================] - 172s 1ms/step - loss: 0.0441 - acc: 0.9835 - val_loss: 0.0470 - val_acc: 0.9828
Epoch 3/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0377 - acc: 0.9853 - val_loss: 0.0470 - val_acc: 0.9825
Epoch 4/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0318 - acc: 0.9873 - val_loss: 0.0521 - val_acc: 0.9818
Epoch 5/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0270 - acc: 0.9892 - val_loss: 0.0584 - val_acc: 0.9817
Epoch 6/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0226 - acc: 0.9910 - val_loss: 0.0619 - val_acc: 0.9806
Epoch 7/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0190 - acc: 0.9925 - val_loss: 0.0709 - val_acc: 0.9810
Epoch 8/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0158 - acc: 0.9939 - val_loss: 0.0766 - val_acc: 0.9805
Epoch 9/10
119678/119678 [=====